# Part 1

In [1]:
# Taken from day 17 and modified for today.
def render_map(map_):
    ''' Render a section of the [infinite] map. '''
    min_x = min(coord[0] for coord in map_.keys())
    max_x = max(coord[0] for coord in map_.keys())
    min_y = min(coord[1] for coord in map_.keys())
    max_y = max(coord[1] for coord in map_.keys())
    render = ''
    for y in range(min_y, max_y+1):
        render += ''.join('X' if x==0 and y==0 else map_.get((x,y),' ') for x in range(min_x, max_x+1))
        render += '\n'
    return render

def print_map(map_):
    print(render_map(map_))

In [2]:
def create_map():
    ''' Create a blank map with the intial room at 0,0. '''
    return {
        (-1,-1): '#', ( 0,-1): '?', ( 1,-1): '#',
        (-1, 0): '?', ( 0, 0): '.', ( 1, 0): '?',
        (-1, 1): '#', ( 0, 1): '?', ( 1, 1): '#',
    }

In [3]:
def fill_walls(map_):
    ''' Replace new map where ? is replaced with #. '''
    return {(x, y):('#' if char == '?' else char) for (x,y), char in map_.items()}

In [4]:
test_map = create_map()
print_map(test_map)
print_map(fill_walls(test_map))

#?#
?X?
#?#

###
#X#
###



In [5]:
class Symbol:
    def __init__(self, name):
        self.name = name
        
    def __repr__(self):
        return 'Symbol({})'.format(self.name)

OPEN = Symbol('OPEN')
PIPE = Symbol('PIPE')
CLOSE = Symbol('CLOSE')
    
def lex_regex(regex):
    ''' Split regex into a sequence of tokens. Ignores start and end specifiers: ^$. '''
    tokens = list()
    current = list()
    for char in regex:
        if char in 'NESW':
            current.append(char)
            continue
        if current:
            tokens.append(''.join(current))
            current = list()
        if char == '(':
            tokens.append(OPEN)
        elif char == '|':
            tokens.append(PIPE)
        elif char == ')':
            tokens.append(CLOSE)
    return tokens

In [6]:
def debug_lex(regex):
    print(lex_regex(regex))
    
debug_lex('^WNE$')

['WNE']


In [7]:
debug_lex('^N(E|W)N$')

['N', Symbol(OPEN), 'E', Symbol(PIPE), 'W', Symbol(CLOSE), 'N']


In [8]:
debug_lex('^NN(EE|WW(SS|NN))EE$')

['NN', Symbol(OPEN), 'EE', Symbol(PIPE), 'WW', Symbol(OPEN), 'SS', Symbol(PIPE), 'NN', Symbol(CLOSE), Symbol(CLOSE), 'EE']


In [85]:
class RegexList(list):
    ''' A list that contains regex strings and groups. '''
    def append(self, x):
        assert isinstance(x, (str, RegexGroup))
        super().append(x)
    
    def __str__(self, indent=''):
        render = '{}RegexList[\n'.format(indent)
        for x in self:
            if isinstance(x, str):
                render += '  {}{}\n'.format(indent, x)
            else:
                render += x.__str__(indent + '  ')
        render += '{}]\n'.format(indent)
        return render
        
class RegexGroup:
    ''' A container for RegexList items. '''
    def __init__(self):
        self._items = list()

    def __str__(self, indent=''):
        render = '{}RegexGroup{{\n'.format(indent)
        for i in self._items:
            render += i.__str__(indent + '  ')
        render += '{}}}\n'.format(indent)
        return render
        
    def add_item(self, item):
        assert isinstance(item, RegexList)
        self._items.append(item)
    
    @property
    def items(self):
        return self._items

In [86]:
rg = RegexGroup()
rl1 = RegexList()
rl1.append('NN')
rg.add_item(rl1)
rl2 = RegexList()
rl2.append('SS')
rg.add_item(rl2)
print(rg)

RegexGroup{
  RegexList[
    NN
  ]
  RegexList[
    SS
  ]
}



In [87]:
rl = RegexList()
rl.append('EE')
rl.append(rg)
rl.append('WW')
print(rl)

RegexList[
  EE
  RegexGroup{
    RegexList[
      NN
    ]
    RegexList[
      SS
    ]
  }
  WW
]



In [88]:
def parse_tokens(tokens):
    ''' Parse tokens and return a RegexList. '''
    # Create a stack of regex lists and a stack of groups
    root = RegexList()
    regex_lists = [root]
    groups = []
    for token in tokens:
        if isinstance(token, Symbol):
            if token is OPEN:
                # Create a new group and add it to current regex list
                group = RegexGroup()
                groups.append(group)
                regex_lists[-1].append(group)
                # Create a new regex list. Add it to group and put it on 
                # the stack
                rl = RegexList()
                regex_lists.append(rl)
                group.add_item(rl)
            elif token is PIPE:
                # We have finished a regex list. Pop it off the stack and create
                # a new regex list.
                regex_list = regex_lists.pop()
                rl = RegexList()
                groups[-1].add_item(rl)
                regex_lists.append(rl)
            elif token is CLOSE:
                # We have finished a regex list. Pop it off the stack.
                regex_list = regex_lists.pop()
                # We have finished a group. Pop it off the stack.
                group = groups.pop()                
        else:
            # Token is a plain string. Append to current list.
            regex_lists[-1].append(token)
    return root

def debug_parse(regex):
    tokens = lex_regex(regex)
    print(tokens)
    regex_list = parse_tokens(tokens)
    print(regex_list)

In [89]:
debug_parse('^NN(EE|WW)SS$')

['NN', Symbol(OPEN), 'EE', Symbol(PIPE), 'WW', Symbol(CLOSE), 'SS']
RegexList[
  NN
  RegexGroup{
    RegexList[
      EE
    ]
    RegexList[
      WW
    ]
  }
  SS
]



In [90]:
debug_parse('^NN(EE|WW(NN|SS)WW)SS$')

['NN', Symbol(OPEN), 'EE', Symbol(PIPE), 'WW', Symbol(OPEN), 'NN', Symbol(PIPE), 'SS', Symbol(CLOSE), 'WW', Symbol(CLOSE), 'SS']
RegexList[
  NN
  RegexGroup{
    RegexList[
      EE
    ]
    RegexList[
      WW
      RegexGroup{
        RegexList[
          NN
        ]
        RegexList[
          SS
        ]
      }
      WW
    ]
  }
  SS
]



In [144]:
def plot_regex_string(regex, map_, x, y):
    ''' Given a regex string that contains only directions (no groups), 
    plot the corresponding walls, doors, rooms, and question marks on 
    the map. Returns updated x,y. '''

    def question(qx, qy):
        ''' If qx,qy not in map, then set it to question mark. '''
        if (qx,qy) not in map_:
            map_[qx,qy] = '?'

    for char in regex:
        if char == 'N':
            map_[x,  y-1] = '-'
            map_[x  ,y-2] = '.'
            map_[x-1,y-3] = '#'
            map_[x+1,y-3] = '#'
            question(x-1, y-2)
            question(x+1, y-2)
            question(x  , y-3)
            y -= 2
        elif char == 'S':
            map_[x,  y+1] = '-'
            map_[x  ,y+2] = '.'
            map_[x-1,y+3] = '#'
            map_[x+1,y+3] = '#'
            question(x-1, y+2)
            question(x+1, y+2)
            question(x  , y+3)
            y += 2
        elif char == 'W':
            map_[x-1,y  ] = '|'
            map_[x-2,y  ] = '.'
            map_[x-3,y-1] = '#'
            map_[x-3,y+1] = '#'
            question(x-2, y-1)
            question(x-2, y+1)
            question(x-3, y  )
            x -= 2
        elif char == 'E':
            map_[x+1,y  ] = '|'
            map_[x+2,y  ] = '.'
            map_[x+3,y-1] = '#'
            map_[x+3,y+1] = '#'
            question(x+2, y-1)
            question(x+2, y+1)
            question(x+3, y  )
            x += 2
        else:
            raise Exception('Unexpected char="{}"'.format(char))
    
    return x, y

def plot_regex_item(item, map_, points):
    ''' Plot a regex item (str or RegexGroup) from each of the given endpoints. 
    Return new endpoints. '''
    new_points = list()
    for x, y in points:
        if isinstance(item, str):
            new_points.append(plot_regex_string(item, map_, x, y))
        elif isinstance(item, RegexGroup):
            for rl in item.items:
                new_points += plot_regex_list(rl, map_, points)
    return new_points

def debug_plot_regex_item(item, points):
    map_ = create_map()
    points = plot_regex_item(item, map_, points)
    print_map(map_)
    return points

def plot_regex_list(regex_list, map_, points):
    ''' Plot each item of a regex list, starting at the given endpoints. Returns
    a list of new endpoints. '''
    for item in regex_list:
        points += plot_regex_item(item, map_, points)
    return points

def plot(regex):
    tokens = lex_regex(regex)
    regex_list = parse_tokens(tokens)
    map_ = create_map()
    plot_regex_list(regex_list, map_, points=[(0,0)])
    print_map(map_)

In [139]:
# Plot N and S.
plot('^NSS$')

#?#
?.?
#-#
?X?
#-#
?.?
#?#



In [140]:
# Plot E and W
plot('^EWW$')

#?#?#?#
?.|X|.?
#?#?#?#



In [146]:
debug_plot_regex_item('NSS', points=[(0,0),(-6,-2)])

#?#      
?.?      
 -    #?#
?.?   ?.?
#-#   #-#
?.?   ?X?
#?#   #-#
      ?.?
      #?#



[(0, 2), (-6, 0)]

In [148]:
rg = RegexGroup()
rl1 = RegexList()
rl1.append('NNN')
rg.add_item(rl1)
rl1 = RegexList()
rl1.append('EEE')
rg.add_item(rl2)
debug_plot_regex_item(rg, points=[(0,0)])

KeyboardInterrupt: 

In [149]:
# Basic tests for groups. This should make a Y shape.
plot('^NN(EE|WW)NN$')

KeyboardInterrupt: 

# Part 2